# Рекурсивные функции

Для того, чтобы можно было охватить также и множество частично определенных фунций, необходимо расширить класс примитивно рекурсивных функций, добавив еще одну операцию для продуцирования новых функций.

Интуитивно, частично определенная функция (*partial function*) $f$ является вычислимой, если может быть задан список определенных и явных инструкций, следуя которым, в случае применения к любому $x$, принадлежащему области определения функции, можно получить значение $f(x)$, однако, в случае применения к $x$, не принадлежащему области определения функции, следование инструкциям будет продолжаться вечно, не приводя к результату, либо завершится не дав результат. Это касается и функций от многих переменных.

Класс общерекурсивных (или просто рекурсивных) функций содержит в себе класс примитивно рекурсивных функций, к которым сводятся всюду определенные функции, и класс частично рекурсивиных функций, к которым сводятся частично определенные функции.

Класс рекурсивных функций получается из класса примитивно рекурсивных функций путем добавления операции минимизации для продуцирования новых функций.

## Минимизация

Пусть дана функция $f$ от $n+1$ аргументов $f(x_1, \ldots, x_n, y)$. Операция **минимизации** (*minimization*) дает всюду или частично определенную функцию $h$ от $n$ аргументов $h(x_1, \ldots, x_n)$:
$$
\mathrm{Mn}[f](x_1, \ldots, x_n) = 
\begin{cases}
y & \text{если } f(x_1, \ldots, x_n, y)=0, \\ 
& \text{и для всех } t<y \\
& f(x_1, \ldots, x_n, t) \text{ определена и не равна нулю} \\
\text{undefined} & \text{если если не существует такого } y
\end{cases}
$$
Функция $h$ возвращает минимальное значение последнего аргумента функции $f$, при котором $f$ принимает значение $0$, при условии, что функция определена при всех значениях последнего аргумента, которые меньше этого минимального значения.

Для $h(\mathbf x)=h(x_1, \ldots, x_n)=\mathrm{Mn}[f]$ мы вычисляем последовательно $f(\mathbf x, 0), f(\mathbf x, 1), f(\mathbf x, 2), \ldots$, останавливаясь при достижении такого $y$, при котором $f(\mathbf x, y) = 0$. Для $\mathbf x$, функция $h$ может оказаться неопределенной по двум причинам. Во-первых, может оказаться, что все значения последовательности $f(\mathbf x, 0), f(\mathbf x, 1), f(\mathbf x, 2), \ldots$ определены, нет нулевых значений. Во-вторых, может оказаться, что значения последовательности $f(\mathbf x, 0), f(\mathbf x, 1), \ldots, f(\mathbf x, i-1)$ определены, но среди них нет нулевых, а $f(\mathbf x, i)$ уже не определена. В этих случаях попытка вычислить $h(\mathbf x)$ будет вовлекать в процесс, который будет продолжнаться вечно, не давая результата. 

В случае, когда $f$ является всюду определенной, мы не беспокоимся о второй причине неопределенности $h$ и определение выще может быть записано так:
$$
\mathrm{Mn}[f](x_1, \ldots, x_n) = 
\begin{cases}
\text{наименьший }y, \text{при котором} \\ \quad f(x_1, \ldots, x_n, y)=0
& \text{если существует такой } y\\ 
\text{undefined} & \text{в противном случае}
\end{cases}
$$

Всюду определенная функция $f$ называется **регулярной** (*regular*), если для каждого $x_1, \ldots, x_n$ существует такой $y$, при котором $f(x_1, \ldots, x_n, y)=0$. В случае, когда $f$ является регулярной функцией, $\mathrm{Mn}[f]$ будет всюду определенной функцией. Более того, если $f$ является всюду определенной функцией, то $\mathrm{Mn}[f]$ будет всюду определенной (при любых $\mathbf x$) функцией в том и только в том случае, когда $f$ является регулярной. В качестве примера можно рассмотреть функции сложения и умножения: умножение является регулярной функцией, а сложение - нет, так как $f(x,y)=x+y=0$ имеет место только при $x=y=0$.

Функции, которые могут быть получены из базисных функций $z, s, I_i^k$ при помощи операций $\mathrm{Cn, Pr, Mn}$ называются рекурсивными функциями (*recursive functions*). В литературе обычно под **рекурсивной функцией** подразумевают всюду определенную рекурсивную функцию, а под **частичной рекурсивной функцией** (*partial recursive function*) подразумевают в общем как частично определенную, так и всюду определенную рекурсивной функции (всюду определенная функция является частным случаем частично определенной функции).

Операция минимизации позволяет получить обратные функции.

## Пример

Определим фунцию $\mathrm{Div}(x, y)$ – целая часть от деления $x$ на $y$. Исходная функция может быть выбрана так:

$$
f(x, y, z) = \left[ y(z+1) < x \right]
$$

Функция всюду определена на $\mathbb N_0$, область значений $\{0, 1\}$. Значение функции будет равно $1$ для всех значений $z$, при которых значение выражения $y(z+1)$ не превышает $x$. Если увеличивать $z$, то как только значение выражения $y(z+1)$ превысит $x$ функция $f$ вернет $0$. Это наименьшее $z$, при котором $f$ обращается в $0$ и будет целой частью от деления $x$ на $y$:

$$
z = \left\lfloor \frac{x}{y} \right\rfloor\
$$

Таким образом,

$$
\mathrm{Div}(x, y) = \mathrm{Mn}[f] = \mathrm{Mn}\left[ y(z+1) < x \right]
$$

Следует отметить, что функции получения целой части от деления и остатка, вообще говоря можно реализовать через примитивно рекурсивные функции.

# Тезис Чёрча
Согласно тезису Чёрча все вычислимые всюду определенные функции являются рекурсивными функциями (гипотиза о том, что все вычислимые частично определенные функции являются рекурсивными, известна как расширенная версия тезиса Чёрча). На сегодняшний день, тезис Чёрча представляет собой просто гипотезу. Правдоподобность тезиса подтверждается значительным числом вычислимых функций, для которых показано, что они являются рекурсивными. Однако это конечно не доказывает правильность тезиса.